In [ ]:
function node_master(node_pool)
    master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # Définiton des variables z_c
    C = size(node_pool,1)
    z = Array{VariableRef,1}(undef,C)
    
    # Ajout des variables au problème maître restreint
    for c in 1:C
        z[c] = @variable(master, lower_bound=0, upper_bound=1) 
        set_name(z[c], "z_$c")
    end
    
    # Ajout de la contrainte : un couple donneur/receveur appartient au plus à un cycle
    cyclechoice = @constraint(master, cyclechoice[i in VERTICES],
                              sum(z[c]*node_pool[c]["vertices"][i] for c in 1:C) <= 1)
    
    # Ajout de l'objectif : maximiser le nombre d'échanges importants de reins
    
    @objective(master, Max, sum(z[c]*node_pool[c]["cost"] for c in 1:C))

    # On renvoie les références des contraintes et des variables ainsi que le modèle
    return cyclechoice, z, master
end